# Container Volume Mounts

A Volume Mount is a way to give a Container a mounted filesystem, specified by a path.

With mounts you achieve:

- Provide data avoiding embedding actual data in the same container
- Reduce overhead and improve storage read/write performance
- Share content

There are 2 kinds of volumes in respect to the host:

- Volume mounts from the host (for example, from macOS to the Podman Machine)
- Named volumes (only available to the container engine)


## Accessing a file using a volume mount

Accessing a file from the host (macOS) within the container requires a volume mount.

Let's create a sample file and see how this works:

In [3]:
# Create a sample folder and file
mkdir -p tests/volume/
echo "volume test, this is accessed by the busybox container" > tests/volume/test

: 1

In [4]:
# Now let's mount the filesystem and access it on the container
podman run -it --rm -v $PWD/tests:/tests  busybox cat /tests/volume/test

volume test, this is accessed by the busybox container


: 1

## Named Volume Mounts

A _named volume mount_ is a mount of a volume by name:
- A volume needs to exist and have a name (needs to be created previous to the volume mount)
- The volume's name is then passed to the left side of the `:` on the volume binding argument
- The container runtime understands each `non-path` argument on the left side of the binding `:` as a named volume.


### Testing persistence through volume binding

We will create a sample volume and run some commands within different containers to fill a file with data.

The containers will be removed after each execution (`--rm` flag), but the data will be persisted.

In [1]:
podman volume create sample-volume

sample-volume


: 1

In [3]:
podman run --rm --name busybox-with-volume -v sample-volume:/data busybox sh -c 'date >> /data/persistent_data.txt && cat /data/persistent_data.txt'

Mon Jul 25 15:48:52 UTC 2022


: 1

In [4]:
podman run --rm --name busybox-with-volume -v sample-volume:/data busybox sh -c 'date >> /data/persistent_data.txt && cat /data/persistent_data.txt'

Mon Jul 25 15:48:52 UTC 2022
Mon Jul 25 15:49:08 UTC 2022


: 1

In [6]:
for c in $(seq 1 10); do
    podman run --rm -v sample-volume:/data busybox sh -c 'date >> /data/persistent_data.txt && cat /data/persistent_data.txt'
done

Mon Jul 25 15:48:52 UTC 2022
Mon Jul 25 15:49:08 UTC 2022
Mon Jul 25 15:52:05 UTC 2022
Mon Jul 25 15:48:52 UTC 2022
Mon Jul 25 15:49:08 UTC 2022
Mon Jul 25 15:52:05 UTC 2022
Mon Jul 25 15:52:06 UTC 2022
Mon Jul 25 15:48:52 UTC 2022
Mon Jul 25 15:49:08 UTC 2022
Mon Jul 25 15:52:05 UTC 2022
Mon Jul 25 15:52:06 UTC 2022
Mon Jul 25 15:52:06 UTC 2022
Mon Jul 25 15:48:52 UTC 2022
Mon Jul 25 15:49:08 UTC 2022
Mon Jul 25 15:52:05 UTC 2022
Mon Jul 25 15:52:06 UTC 2022
Mon Jul 25 15:52:06 UTC 2022
Mon Jul 25 15:52:07 UTC 2022
Mon Jul 25 15:48:52 UTC 2022
Mon Jul 25 15:49:08 UTC 2022
Mon Jul 25 15:52:05 UTC 2022
Mon Jul 25 15:52:06 UTC 2022
Mon Jul 25 15:52:06 UTC 2022
Mon Jul 25 15:52:07 UTC 2022
Mon Jul 25 15:52:07 UTC 2022
Mon Jul 25 15:48:52 UTC 2022
Mon Jul 25 15:49:08 UTC 2022
Mon Jul 25 15:52:05 UTC 2022
Mon Jul 25 15:52:06 UTC 2022
Mon Jul 25 15:52:06 UTC 2022
Mon Jul 25 15:52:07 UTC 2022
Mon Jul 25 15:52:07 UTC 2022
Mon Jul 25 15:52:08 UTC 2022
Mon Jul 25 15:48:52 UTC 2022
Mon Jul 25 15:

: 1

### Clean up the sample volume

Note: Deleting a volume erases the data within.

In [7]:
podman volume rm sample-volume

sample-volume


: 1